In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 8), abs(state[i*4+1] + state[i*4+3] * 8), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(8)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[ 0.10595895  0.07077589  0.06184354  0.06426361]
 [-0.88543385 -1.02056652 -0.21745695 -0.69002132]
 [ 0.08235181 -0.88485404  0.0227449  -0.17054959]
 ...
 [ 0.17919906  0.19869572  0.03130001  0.19672003]
 [ 0.22799573  0.19742117  0.10831008  0.30209988]
 [-0.63263955 -0.78469977 -0.73861277 -0.03337042]]


In [5]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action)
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0

print("Training finished.\n")

Episode: 100
average reward: -145.27
Episode: 200
average reward: -28.88
Episode: 300
average reward: -101.74
Episode: 400
average reward: -67.1
Episode: 500
average reward: -41.85
Episode: 600
average reward: -89.8
Episode: 700
average reward: -112.69
Episode: 800
average reward: -55.72
Episode: 900
average reward: -25.42
Episode: 1000
average reward: -36.74
Episode: 1100
average reward: -100.56
Episode: 1200
average reward: -75.54
Episode: 1300
average reward: -64.42
Episode: 1400
average reward: -43.09
Episode: 1500
average reward: -17.02
Episode: 1600
average reward: -32.55
Episode: 1700
average reward: -60.25
Episode: 1800
average reward: -56.35
Episode: 1900
average reward: -43.7
Episode: 2000
average reward: -56.94
Episode: 2100
average reward: -54.71
Episode: 2200
average reward: -15.36
Episode: 2300
average reward: -55.71
Episode: 2400
average reward: -67.47
Episode: 2500
average reward: -18.87
Episode: 2600
average reward: -47.53
Episode: 2700
average reward: 11.07
Episode: 2

Episode: 22100
average reward: -37.92
Episode: 22200
average reward: -30.52
Episode: 22300
average reward: -86.98
Episode: 22400
average reward: -61.35
Episode: 22500
average reward: -2.37
Episode: 22600
average reward: -10.72
Episode: 22700
average reward: -90.06
Episode: 22800
average reward: -94.19
Episode: 22900
average reward: -11.73
Episode: 23000
average reward: -18.75
Episode: 23100
average reward: -81.8
Episode: 23200
average reward: -59.86
Episode: 23300
average reward: -51.89
Episode: 23400
average reward: -11.34
Episode: 23500
average reward: -49.53
Episode: 23600
average reward: -101.71
Episode: 23700
average reward: -99.23
Episode: 23800
average reward: -81.08
Episode: 23900
average reward: -9.6
Episode: 24000
average reward: -29.5
Episode: 24100
average reward: -25.15
Episode: 24200
average reward: -72.87
Episode: 24300
average reward: 25.89
Episode: 24400
average reward: 1.71
Episode: 24500
average reward: -87.61
Episode: 24600
average reward: -66.53
Episode: 24700
aver

Episode: 43800
average reward: -40.81
Episode: 43900
average reward: -41.89
Episode: 44000
average reward: -61.47
Episode: 44100
average reward: -79.14
Episode: 44200
average reward: -71.53
Episode: 44300
average reward: -43.6
Episode: 44400
average reward: -79.25
Episode: 44500
average reward: -107.44
Episode: 44600
average reward: -46.63
Episode: 44700
average reward: -25.06
Episode: 44800
average reward: -50.59
Episode: 44900
average reward: -21.59
Episode: 45000
average reward: -75.84
Episode: 45100
average reward: -26.29
Episode: 45200
average reward: -24.21
Episode: 45300
average reward: -32.15
Episode: 45400
average reward: -65.81
Episode: 45500
average reward: -60.63
Episode: 45600
average reward: -58.28
Episode: 45700
average reward: -10.93
Episode: 45800
average reward: -73.67
Episode: 45900
average reward: -73.62
Episode: 46000
average reward: -82.75
Episode: 46100
average reward: -21.08
Episode: 46200
average reward: -100.84
Episode: 46300
average reward: -54.99
Episode: 46

Episode: 65500
average reward: -67.34
Episode: 65600
average reward: 7.45
Episode: 65700
average reward: -28.92
Episode: 65800
average reward: -71.84
Episode: 65900
average reward: -44.24
Episode: 66000
average reward: 24.09
Episode: 66100
average reward: 9.27
Episode: 66200
average reward: -21.32
Episode: 66300
average reward: -24.76
Episode: 66400
average reward: -83.53
Episode: 66500
average reward: -41.83
Episode: 66600
average reward: -28.92
Episode: 66700
average reward: 15.24
Episode: 66800
average reward: -9.19
Episode: 66900
average reward: -47.32
Episode: 67000
average reward: -34.52
Episode: 67100
average reward: -43.2
Episode: 67200
average reward: -30.45
Episode: 67300
average reward: -85.85
Episode: 67400
average reward: -93.72
Episode: 67500
average reward: -34.41
Episode: 67600
average reward: -46.71
Episode: 67700
average reward: -120.18
Episode: 67800
average reward: -55.49
Episode: 67900
average reward: -73.5
Episode: 68000
average reward: -40.66
Episode: 68100
avera

Episode: 87200
average reward: -59.53
Episode: 87300
average reward: 34.97
Episode: 87400
average reward: -45.22
Episode: 87500
average reward: -39.6
Episode: 87600
average reward: -44.7
Episode: 87700
average reward: -83.39
Episode: 87800
average reward: -49.42
Episode: 87900
average reward: -12.43
Episode: 88000
average reward: -5.02
Episode: 88100
average reward: -59.8
Episode: 88200
average reward: -67.6
Episode: 88300
average reward: -58.25
Episode: 88400
average reward: -5.9
Episode: 88500
average reward: -84.04
Episode: 88600
average reward: -93.04
Episode: 88700
average reward: -64.27
Episode: 88800
average reward: -116.25
Episode: 88900
average reward: -79.8
Episode: 89000
average reward: 16.67
Episode: 89100
average reward: -68.23
Episode: 89200
average reward: -53.97
Episode: 89300
average reward: -6.92
Episode: 89400
average reward: -47.25
Episode: 89500
average reward: -4.66
Episode: 89600
average reward: -45.05
Episode: 89700
average reward: -56.76
Episode: 89800
average 

In [6]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [7]:
import time
total_epochs, total_penalties = 0, 0
episodes = 10
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 10 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 109.4
Average reward per episode: 1.2
